## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,2022-02-13 22:03:01,-0.7393,-90.3518,76.55,89,90,1.01,overcast clouds
1,1,Busselton,AU,2022-02-13 22:06:13,-33.6500,115.3333,63.05,77,0,11.72,clear sky
2,2,Alta Floresta,BR,2022-02-13 22:06:14,-9.8756,-56.0861,73.60,94,100,2.15,overcast clouds
3,3,Coquimbo,CL,2022-02-13 22:06:14,-29.9533,-71.3436,69.53,64,0,12.66,clear sky
4,4,Arlit,NE,2022-02-13 22:06:15,18.7369,7.3853,67.96,12,0,5.37,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,2022-02-13 22:03:01,-0.7393,-90.3518,76.55,89,90,1.01,overcast clouds
18,18,Rikitea,PF,2022-02-13 22:06:20,-23.1203,-134.9692,78.51,71,74,12.59,light rain
20,20,Georgetown,MY,2022-02-13 22:05:08,5.4112,100.3354,79.90,86,20,6.91,thunderstorm
23,23,Oistins,BB,2022-02-13 22:01:32,13.0667,-59.5333,77.88,69,48,17.85,scattered clouds
26,26,Vaini,TO,2022-02-13 22:06:22,-21.2000,-175.2000,80.71,75,88,19.06,overcast clouds
30,30,Ahuimanu,US,2022-02-13 22:06:24,21.4447,-157.8378,80.31,62,40,13.80,scattered clouds
34,34,Atuona,PF,2022-02-13 22:06:25,-9.8000,-139.0333,77.97,78,22,12.91,light rain
38,38,Hilo,US,2022-02-13 22:05:53,19.7297,-155.0900,78.53,70,40,8.05,scattered clouds
40,40,General Pico,AR,2022-02-13 22:06:28,-35.6566,-63.7568,75.51,37,0,7.07,clear sky
49,49,Maryborough,AU,2022-02-13 22:07:31,-25.5333,152.7000,75.02,77,28,9.93,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                197
City                   197
Country                197
Date                   197
Lat                    197
Lng                    197
Max Temp               197
Humidity               197
Cloudiness             197
Wind Speed             197
Current Description    197
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,76.55,overcast clouds,-0.7393,-90.3518,
18,Rikitea,PF,78.51,light rain,-23.1203,-134.9692,
20,Georgetown,MY,79.90,thunderstorm,5.4112,100.3354,
23,Oistins,BB,77.88,scattered clouds,13.0667,-59.5333,
26,Vaini,TO,80.71,overcast clouds,-21.2000,-175.2000,
30,Ahuimanu,US,80.31,scattered clouds,21.4447,-157.8378,
34,Atuona,PF,77.97,light rain,-9.8000,-139.0333,
38,Hilo,US,78.53,scattered clouds,19.7297,-155.0900,
40,General Pico,AR,75.51,clear sky,-35.6566,-63.7568,
49,Maryborough,AU,75.02,scattered clouds,-25.5333,152.7000,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,76.55,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
18,Rikitea,PF,78.51,light rain,-23.1203,-134.9692,People ThankYou
20,Georgetown,MY,79.90,thunderstorm,5.4112,100.3354,Cititel Penang
23,Oistins,BB,77.88,scattered clouds,13.0667,-59.5333,Butterfly Beach Hotel
26,Vaini,TO,80.71,overcast clouds,-21.2000,-175.2000,Keleti Beach Resort


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))